In [5]:
from pyspark.sql import SparkSession
# spark session is the enrty point to the cluster 
spark = SparkSession. \
builder.\
master("spark://spark-master:7077")\
.config("spark.sql.warehouse.dir", "/data/spark-warehouse") \
.config("spark.sql.catalogImplementation", "hive") \
.config("javax.jdo.option.ConnectionURL", "jdbc:derby:/data/metastore_db;create=true")\
.appName("dataframe").getOrCreate()

In [4]:
spark.stop()

In [47]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
orders_schema_struct = StructType(
[
StructField("order_id",IntegerType(),False)
,StructField("customer_id",IntegerType(),False)
,StructField("product_id",IntegerType(),False)
,StructField("unit_price",FloatType(),False)
,StructField("order_date",DateType(),False)
,StructField("order_status",StringType(),False)
,StructField("state",StringType(),False)
,StructField("quantity",IntegerType(),False)
]   
)
# 612 mb , 4 cores
df_orders = spark.read.csv("/data/orders_600mb.csv" , header = True ,  schema = orders_schema_struct)

product_schema =  'product_id int , product_name string, cost_price integer'

df_products = spark.read.csv("/data/products.csv" , header = True ,  schema = product_schema)
df_orders.show()
df_products.show()

+--------+-----------+----------+----------+----------+------------+--------------+--------+
|order_id|customer_id|product_id|unit_price|order_date|order_status|         state|quantity|
+--------+-----------+----------+----------+----------+------------+--------------+--------+
|       1|     259211|        88|    188.36|2003-10-22|      PLACED|        Kerala|       2|
|       2|     721234|        45|    180.61|2022-05-22|      PLACED|   West Bengal|       3|
|       3|     762837|        29|    183.49|2021-05-30|      PLACED|        Punjab|       1|
|       4|     726253|        42|    181.44|2011-04-07|      PLACED|         Bihar|       3|
|       5|     396288|         2|    108.84|2018-03-22|    RETURNED|   Uttarakhand|       1|
|       6|     226571|        17|    122.08|2012-02-26|   DELIVERED| Uttar Pradesh|       1|
|       7|     328160|        60|    170.59|2020-10-15|     SHIPPED|       Haryana|       1|
|       8|     259916|        35|    116.47|2007-06-20|     SHIPPED|Ma

In [5]:
def order_priority(order_status, state, unit_price, quantity):
    order_value = unit_price * quantity

    if order_status == "CANCELLED":
        return "IGNORE"

    if state in ("Delhi", "Haryana", "Punjab"):
        if order_value >= 300:
            return "P1"
        else:
            return "P2"
    else:
        if order_value >= 500:
            return "P1"
        else:
            return "P3"


order_priority_udf = udf(order_priority, StringType())

In [7]:
df_orders.withColumn(
    "order_priority",
    order_priority_udf(
        col("order_status"),
        col("state"),
        col("unit_price"),
        col("quantity")
    )
).write.mode("overwrite").format("noop").save()

In [8]:
df_orders.createOrReplaceTempView("orders")

spark.sql("select * from orders").show()

+--------+-----------+----------+----------+----------+------------+--------------+--------+
|order_id|customer_id|product_id|unit_price|order_date|order_status|         state|quantity|
+--------+-----------+----------+----------+----------+------------+--------------+--------+
|       1|     997568|        16|    148.27|2022-09-19|     SHIPPED|         Assam|       1|
|       2|     980660|        90|    193.15|2019-07-20|     SHIPPED|Madhya Pradesh|       1|
|       3|     293603|        47|    117.13|2006-11-16|      PLACED|         Delhi|       2|
|       4|      88784|        40|    169.87|2021-12-20|   DELIVERED|     Karnataka|       3|
|       5|     639307|        89|    125.48|2022-12-12|   DELIVERED|     Jharkhand|       1|
|       6|     460431|        83|     179.2|2000-01-17|     SHIPPED|     Rajasthan|       1|
|       7|     197087|        14|    185.22|2019-09-21|     SHIPPED|        Punjab|       3|
|       8|     881816|         8|    161.35|2012-10-13|      PLACED|  

In [5]:
spark.sql("show databases").show()

26/01/24 06:18:23 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/01/24 06:18:23 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
26/01/24 06:18:25 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
26/01/24 06:18:25 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.5
26/01/24 06:18:25 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


+---------+
|namespace|
+---------+
|  default|
+---------+



In [6]:
spark.sql("create database mydb")

26/01/24 06:18:30 WARN ObjectStore: Failed to get database mydb, returning NoSuchObjectException
26/01/24 06:18:30 WARN ObjectStore: Failed to get database mydb, returning NoSuchObjectException
26/01/24 06:18:30 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
26/01/24 06:18:30 WARN ObjectStore: Failed to get database mydb, returning NoSuchObjectException


DataFrame[]

In [8]:
spark.sql("create table mydb.emp_p (id int , salary int) USING PARQUET")

26/01/24 06:19:12 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
26/01/24 06:19:12 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
26/01/24 06:19:12 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/01/24 06:19:12 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


DataFrame[]

In [9]:
spark.sql("insert into mydb.emp_p values (2,2000)")

DataFrame[]

26/01/24 06:19:42 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-848d82a5-ddd6-48e4-9c14-44480c7fe991/userFiles-eb68e02d-264d-4cad-8d01-ff452cb65418. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-848d82a5-ddd6-48e4-9c14-44480c7fe991/userFiles-eb68e02d-264d-4cad-8d01-ff452cb65418
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.SparkEnv.stop(SparkEnv.scala:108)
	at org.apache.spark.SparkContext.$anonfun$stop$25(SparkContext.scala:2310)
	at org.apac

In [2]:
spark.sql("describe extended mydb.emp_p").show(truncate=False)

26/01/24 06:20:18 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/01/24 06:20:18 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
26/01/24 06:20:19 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
26/01/24 06:20:19 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.5
26/01/24 06:20:19 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|id                          |int                                                           |NULL   |
|salary                      |int                                                           |NULL   |
|                            |                                                              |       |
|# Detailed Table Information|                                                              |       |
|Catalog                     |spark_catalog                                                 |       |
|Database                    |mydb                                                          |       |
|Table                       |emp_p                                               

In [12]:
df_orders.write.mode("overwrite").saveAsTable("orders")


In [7]:
#spark.sql("select * from orders").show()

df = spark.table("orders")
df.show()

+--------+-----------+----------+----------+----------+------------+--------------+--------+
|order_id|customer_id|product_id|unit_price|order_date|order_status|         state|quantity|
+--------+-----------+----------+----------+----------+------------+--------------+--------+
|       1|     997568|        16|    148.27|2022-09-19|     SHIPPED|         Assam|       1|
|       2|     980660|        90|    193.15|2019-07-20|     SHIPPED|Madhya Pradesh|       1|
|       3|     293603|        47|    117.13|2006-11-16|      PLACED|         Delhi|       2|
|       4|      88784|        40|    169.87|2021-12-20|   DELIVERED|     Karnataka|       3|
|       5|     639307|        89|    125.48|2022-12-12|   DELIVERED|     Jharkhand|       1|
|       6|     460431|        83|     179.2|2000-01-17|     SHIPPED|     Rajasthan|       1|
|       7|     197087|        14|    185.22|2019-09-21|     SHIPPED|        Punjab|       3|
|       8|     881816|         8|    161.35|2012-10-13|      PLACED|  

In [8]:
spark.sql("drop table orders")

DataFrame[]

In [20]:
spark.sql("describe extended orders_ext").show(truncate=False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|id                          |int                                                           |NULL   |
|amount                      |double                                                        |NULL   |
|                            |                                                              |       |
|# Detailed Table Information|                                                              |       |
|Catalog                     |spark_catalog                                                 |       |
|Database                    |default                                                       |       |
|Table                       |orders_ext                                          

In [18]:
spark.sql("""CREATE TABLE orders_ext (
  id INT,
  amount DOUBLE
)
USING PARQUET
LOCATION '/data/orders_external' """)

26/01/24 06:34:18 WARN HadoopFSUtils: The directory file:/data/orders_external was not found. Was it deleted very recently?


DataFrame[]

In [19]:
spark.sql("insert into orders_ext values (1,100)")

DataFrame[]

In [21]:
spark.sql("drop table orders_ext")

DataFrame[]

In [22]:
df_orders.write \
  .format("parquet") \
  .option("path", "/data/orders_external_df/") \
  .saveAsTable("t_orders_ext")

In [25]:
spark.sql("create table orders_data_ext using parquet location '/data/orders_external_df/'  ")


DataFrame[]

In [26]:
spark.sql("select * from orders_data_ext").show()

+--------+-----------+----------+----------+----------+------------+--------------+--------+
|order_id|customer_id|product_id|unit_price|order_date|order_status|         state|quantity|
+--------+-----------+----------+----------+----------+------------+--------------+--------+
|       1|     997568|        16|    148.27|2022-09-19|     SHIPPED|         Assam|       1|
|       2|     980660|        90|    193.15|2019-07-20|     SHIPPED|Madhya Pradesh|       1|
|       3|     293603|        47|    117.13|2006-11-16|      PLACED|         Delhi|       2|
|       4|      88784|        40|    169.87|2021-12-20|   DELIVERED|     Karnataka|       3|
|       5|     639307|        89|    125.48|2022-12-12|   DELIVERED|     Jharkhand|       1|
|       6|     460431|        83|     179.2|2000-01-17|     SHIPPED|     Rajasthan|       1|
|       7|     197087|        14|    185.22|2019-09-21|     SHIPPED|        Punjab|       3|
|       8|     881816|         8|    161.35|2012-10-13|      PLACED|  

In [28]:
df_orders.write.mode("overwrite").save("/data/orders")

In [45]:
df_orders.write.partitionBy("order_status","state").mode("overwrite").save("/data/orders")

In [32]:
df = spark.read.load("/data/orders")

In [43]:
df.filter(col("state") == 'Karnataka' ).write.format("noop").mode("overwrite").save()

In [42]:
df_orders.filter(col("state") == 'Karnataka' ).write.format("noop").mode("overwrite").save()

In [ ]:
order_status 10-15

order_date

less number of distinct values
high cardinality -> more number of distinct values

In [49]:
# 1.1gb / 128 mb -> 8.3 -> 8 * 128 , 50mb
df_orders.rdd.getNumPartitions()

4.78125

In [50]:
df_orders.repartition(50).write.mode("overwrite").format("csv").save("/data/multiple_files/")

In [51]:
orders_schema_struct = StructType(
[
StructField("order_id",IntegerType(),False)
,StructField("customer_id",IntegerType(),False)
,StructField("product_id",IntegerType(),False)
,StructField("unit_price",FloatType(),False)
,StructField("order_date",DateType(),False)
,StructField("order_status",StringType(),False)
,StructField("state",StringType(),False)
,StructField("quantity",IntegerType(),False)
]   
)
# 612 mb , 4 cores
df_orders_new = spark.read.csv("/data/multiple_files/",  schema = orders_schema_struct)

In [53]:
df_orders_new.rdd.getNumPartitions()

7

In [54]:
50*12.3

615.0

In [58]:
#spark.conf.get("spark.sql.files.openCostInBytes")
4194304/(1024*1028)

3.9844357976653697

In [60]:
(612 + 50*4)/128

6.34375

In [ ]:
200 mb

In [ ]:
karnataka 
goa
UP

create view orders_tech as 
select * from orders where category="Technology"

In [ ]:
table schema orderid int 

In [6]:
spark.sql("show databases").show()

26/01/24 07:54:28 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/01/24 07:54:28 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


+---------+
|namespace|
+---------+
|  default|
|     mydb|
+---------+



26/01/24 07:54:29 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
26/01/24 07:54:29 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.6


In [ ]:
scala spark

In [ ]:
spark-submit code.py